In [1]:
import openai
import numpy as np
from datasets import load_dataset
from tqdm import tqdm

In [4]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12

In [3]:
def get_logprobs_for_choices(question_text, choices=('A', 'B', 'C', 'D'), few_shot_prompt=None):
    prompt = ""
    if few_shot_prompt:
        prompt += few_shot_prompt.strip() + "\n\n"
    prompt += question_text.strip() + "\nAnswer:"

    response = openai.Completion.create(
        model="gpt-4o-mini-2024-07-18",
        prompt=prompt,
        max_tokens=1,
        temperature=0.0,
        logprobs=len(choices),
        echo=False,
    )

    top_logprobs = response["choices"][0]["logprobs"]["top_logprobs"][0]
    cleaned_logprobs = {k.strip(): v for k, v in top_logprobs.items() if k.strip() in choices}
    return cleaned_logprobs


def normalize_logprobs(logprobs_dict):
    keys = list(logprobs_dict.keys())
    values = np.array([logprobs_dict[k] for k in keys])
    probs = np.exp(values - values.max())
    probs /= probs.sum()
    return dict(zip(keys, probs))

dataset = load_dataset("allenai/ai2_arc", "ARC-Challenge", split="train")


def format_arc_question(example):
    choices_str = "\n".join([f"({c}) {example['choices']['text'][i]}" for i, c in enumerate(example['choices']['label'])])
    prompt = f"Question: {example['question']}\nOptions:\n{choices_str}"
    return prompt

README.md: 0.00B [00:00, ?B/s]

ARC-Challenge/train-00000-of-00001.parqu(…):   0%|          | 0.00/190k [00:00<?, ?B/s]

ARC-Challenge/test-00000-of-00001.parque(…):   0%|          | 0.00/204k [00:00<?, ?B/s]

ARC-Challenge/validation-00000-of-00001.(…):   0%|          | 0.00/55.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1119 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1172 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/299 [00:00<?, ? examples/s]

In [7]:
from openai import OpenAI
import numpy as np
from datasets import load_dataset
from tqdm import tqdm


def get_logprobs_for_choices(question_text, choices=('A', 'B', 'C', 'D'), few_shot_prompt=None):
    prompt = ""
    if few_shot_prompt:
        prompt += few_shot_prompt.strip() + "\n\n"
    prompt += question_text.strip() + "\nAnswer:"

    response = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=1,
        temperature=0.0,
        logprobs=len(choices),
        echo=False,
    )

    top_logprobs = response.choices[0].logprobs.top_logprobs[0]
    cleaned_logprobs = {k.strip(): v for k, v in top_logprobs.items() if k.strip() in choices}
    return cleaned_logprobs


def normalize_logprobs(logprobs_dict):
    keys = list(logprobs_dict.keys())
    values = np.array([logprobs_dict[k] for k in keys])
    probs = np.exp(values - values.max())
    probs /= probs.sum()
    return dict(zip(keys, probs))


# ARC-Challenge (train для теста)
dataset = load_dataset("allenai/ai2_arc", "ARC-Challenge", split="test")


def format_arc_question(example):
    choices_str = "\n".join([f"({c}) {example['choices']['text'][i]}" for i, c in enumerate(example['choices']['label'])])
    prompt = f"Question: {example['question']}\nOptions:\n{choices_str}"
    return prompt


correct = 0
total = 0
results = []

for example in tqdm(dataset, desc="Evaluating ARC-Challenge"):
    prompt = format_arc_question(example)
    correct_answer = example["answerKey"]

    try:
        logprobs = get_logprobs_for_choices(prompt)
        norm_probs = normalize_logprobs(logprobs)

        pred_answer = max(norm_probs.items(), key=lambda x: x[1])[0]

        correct += int(pred_answer == correct_answer)
        total += 1

        results.append({
            "question_id": example["id"],
            "correct_answer": correct_answer,
            "predicted_answer": pred_answer,
            "probabilities": norm_probs,
            "raw_logprobs": logprobs,
        })

    except Exception as e:
        print(f"Error on question {example['id']}: {e}")

if total > 0:
    accuracy = correct / total * 100
    print(f"\nARC-Challenge Accuracy (greedy logprobs): {accuracy:.2f}%")
else:
    print("\nERROR: No examples processed.")


Evaluating ARC-Challenge:   0%|          | 1/1119 [00:01<18:59,  1.02s/it]

Error on question Mercury_SC_415702: zero-size array to reduction operation maximum which has no identity


Evaluating ARC-Challenge:   2%|▏         | 17/1119 [00:09<08:01,  2.29it/s]

Error on question NYSEDREGENTS_2004_8_31: zero-size array to reduction operation maximum which has no identity


Evaluating ARC-Challenge:  11%|█         | 124/1119 [00:45<06:10,  2.68it/s]

...

Error on question CSZ20740: zero-size array to reduction operation maximum which has no identity


Evaluating ARC-Challenge:  97%|█████████▋| 1087/1119 [06:15<00:09,  3.24it/s]

Error on question NYSEDREGENTS_2007_8_6: zero-size array to reduction operation maximum which has no identity


Evaluating ARC-Challenge: 100%|██████████| 1119/1119 [06:27<00:00,  2.89it/s]


ARC-Challenge Accuracy (greedy logprobs): 94.25%


In [11]:
import json

In [17]:
def get_logprobs_for_choices(question_text, choices=('A', 'B', 'C', 'D'), few_shot_prompt=None):
    prompt = ""
    if few_shot_prompt:
        prompt += few_shot_prompt.strip() + "\n\n"
    prompt += question_text.strip() + "\nAnswer:"

    response = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=1,
        temperature=0.0,
        logprobs=len(choices),
        echo=False,
    )

    top_logprobs = response.choices[0].logprobs.top_logprobs[0]
    cleaned_logprobs = {k.strip(): v for k, v in top_logprobs.items() if k.strip() in choices}
    if not cleaned_logprobs:
        raise ValueError("No valid A/B/C/D logprobs returned by the model.")
    return cleaned_logprobs


def normalize_logprobs(logprobs_dict):
    keys = list(logprobs_dict.keys())
    values = np.array([logprobs_dict[k] for k in keys])
    probs = np.exp(values - values.max())
    probs /= probs.sum()
    return dict(zip(keys, probs))


def format_arc_question(example):
    choices_str = "\n".join([f"({c}) {example['choices']['text'][i]}" for i, c in enumerate(example['choices']['label'])])
    prompt = f"Question: {example['question']}\nOptions:\n{choices_str}"
    return prompt

In [18]:
few_shot_examples = [
    """Question: What color is the sky on a clear day?
Options:
(A) Green
(B) Blue
(C) Red
(D) Yellow
Answer: B""",
    """Question: What do fish need to breathe?
Options:
(A) Oxygen
(B) Nitrogen
(C) Helium
(D) Hydrogen
Answer: A"""
]


In [19]:
def run_eval(few_shot_count=0, save_to_jsonl="arc_logprobs_results.jsonl"):
    dataset = load_dataset("allenai/ai2_arc", "ARC-Challenge", split="test")
    few_shot_prompt = "\n\n".join(few_shot_examples[:few_shot_count]) if few_shot_count > 0 else None

    correct = 0
    total = 0
    avg_confidence = 0
    results = []

    for example in tqdm(dataset, desc=f"Few-shot {few_shot_count}"):
        prompt = format_arc_question(example)
        correct_answer = example["answerKey"]

        try:
            logprobs = get_logprobs_for_choices(prompt, few_shot_prompt=few_shot_prompt)
            norm_probs = normalize_logprobs(logprobs)

            confidence = norm_probs.get(correct_answer, 0)
            avg_confidence += confidence

            pred_answer = max(norm_probs.items(), key=lambda x: x[1])[0]

            correct += int(pred_answer == correct_answer)
            total += 1

            results.append({
                "question_id": example["id"],
                "correct_answer": correct_answer,
                "predicted_answer": pred_answer,
                "confidence_in_correct": confidence,
                "probabilities": norm_probs,
                "raw_logprobs": logprobs,
            })

        except Exception as e:
            print(f"Skipping {example['id']}: {e}")
            continue

    avg_confidence /= total
    accuracy = correct / total * 100
    print(f"\nFew-shot {few_shot_count} | Accuracy: {accuracy:.2f}% | Avg confidence in correct answer: {avg_confidence:.2f}")

    with open(save_to_jsonl, "w") as f:
        for r in results:
            f.write(json.dumps(r) + "\n")

    print(f"\nSaved results to {save_to_jsonl}")


# Запусти на 0-shot, 1-shot, 2-shot
for few_shot_count in [0, 1, 2]:
    run_eval(few_shot_count=few_shot_count, save_to_jsonl=f"arc_logprobs_results_{few_shot_count}shot.jsonl")


Few-shot 0:   4%|▍         | 45/1172 [00:31<13:08,  1.43it/s]

Skipping NYSEDREGENTS_2008_8_34: No valid A/B/C/D logprobs returned by the model.


Few-shot 0:  11%|█         | 129/1172 [01:29<12:27,  1.39it/s]

Skipping NYSEDREGENTS_2008_8_11: No valid A/B/C/D logprobs returned by the model.


Few-shot 0:  12%|█▏        | 136/1172 [01:34<12:45,  1.35it/s]

...

Skipping NYSEDREGENTS_2010_8_42: No valid A/B/C/D logprobs returned by the model.


Few-shot 2:  98%|█████████▊| 1153/1172 [13:22<00:14,  1.35it/s]

Skipping NYSEDREGENTS_2015_8_29: No valid A/B/C/D logprobs returned by the model.


Few-shot 2: 100%|██████████| 1172/1172 [13:34<00:00,  1.44it/s]


Few-shot 2 | Accuracy: 63.50% | Avg confidence in correct answer: 0.63

Saved results to arc_logprobs_results_2shot.jsonl
